 **Étape 1 : Importer les bibliothèques et les fichiers**

In [1]:
import pandas as pd

# Définir les chemins
fichier_valeurs = "C:/Users/aylin/Downloads/Données-immo/Valeurs-foncières.xlsx"
fichier_geo = "C:/Users/aylin/Downloads/Données-immo/fr-esr-referentiel-geographique.xlsx"
fichier_communes = "C:/Users/aylin/Downloads/Données-immo/donnees_communes.xlsx"

# Charger les fichiers
valeurs_df = pd.read_excel(fichier_valeurs)
geo_df = pd.read_excel(fichier_geo)
communes_df = pd.read_excel(fichier_communes)


**Étape 2 : Créer l'identifiant géographique (id_codedep_codecommune)**

In [4]:
# 1.1 Créer la clé géographique
valeurs_df["Code departement"] = valeurs_df["Code departement"].astype(str).str.zfill(2)
valeurs_df["Code commune"]    = valeurs_df["Code commune"].astype(str).str.zfill(3)
valeurs_df["id_codedep_codecommune"] = (
    valeurs_df["Code departement"] + valeurs_df["Code commune"]
)

 **Étape 3 : Nettoyer les codes dans les autres fichiers (geo_df, communes_df)**

In [7]:
communes_df

,CODREG,CODDEP,CODARR,CODCAN,CODCOM,COM,PMUN,PCAP,PTOT
0,84,01,2.0,08,1,L'Abergement-Clémenciat,779,19,798
1,84,01,1.0,01,2,L'Abergement-de-Varey,256,1,257
2,84,01,1.0,01,4,Ambérieu-en-Bugey,14134,380,14514
3,84,01,2.0,22,5,Ambérieux-en-Dombes,1751,25,1776
4,84,01,1.0,04,6,Ambléon,112,6,118
...,...,...,...,...,...,...,...,...,...
34986,4,974,1.0,04,420,Sainte-Suzanne,24065,227,24292
34987,4,974,3.0,06,421,Salazie,7136,73,7209
34988,4,974,2.0,99,422,Le Tampon,79824,1009,80833
34989,4,974,4.0,14,423,Les Trois-Bassins,7015,91,7106


In [9]:
# Nettoyer et formater les codes
# On convertit les codes de département en chaînes de caractères sur 2 chiffres
geo_df["dep_code"] = geo_df["dep_code"].astype(str).str.zfill(2)

# On convertit les codes de commune en chaînes de caractères sur 3 chiffres                                                             
geo_df["com_code"] = geo_df["com_code"].astype(str).str.zfill(3)

# On concatène les deux pour créer une clé unique "département + commune"
geo_df["id_codedep_codecommune"] = geo_df["dep_code"] + geo_df["com_code"]

# Même traitement pour le fichier des communes (communes_df)
communes_df["CODDEP"] = communes_df["CODDEP"].astype(str).str.zfill(2)
communes_df["CODCOM"] = communes_df["CODCOM"].astype(str).str.zfill(3)
communes_df["id_codedep_codecommune"] = communes_df["CODDEP"] + communes_df["CODCOM"]

# Même chose pour le fichier valeurs foncières (valeurs_df)
valeurs_df["Code departement"] = valeurs_df["Code departement"].astype(str).str.zfill(2)
valeurs_df["Code commune"] = valeurs_df["Code commune"].astype(str).str.zfill(3)
valeurs_df["id_codedep_codecommune"] = valeurs_df["Code departement"] + valeurs_df["Code commune"]



**Étape 4 : Jointure entre geo_df et communes_df**

In [12]:
communes_df=communes_df[["id_codedep_codecommune","CODREG","CODCOM","CODDEP","PTOT","COM"]]

In [14]:
communes_df

,id_codedep_codecommune,CODREG,CODCOM,CODDEP,PTOT,COM
0,01001,84,001,01,798,L'Abergement-Clémenciat
1,01002,84,002,01,257,L'Abergement-de-Varey
2,01004,84,004,01,14514,Ambérieu-en-Bugey
3,01005,84,005,01,1776,Ambérieux-en-Dombes
4,01006,84,006,01,118,Ambléon
...,...,...,...,...,...,...
34986,974420,4,420,974,24292,Sainte-Suzanne
34987,974421,4,421,974,7209,Salazie
34988,974422,4,422,974,80833,Le Tampon
34989,974423,4,423,974,7106,Les Trois-Bassins


In [16]:



# On sélectionne certaines colonnes de geo_df et on fait une jointure avec communes_df
commune_df = geo_df[["id_codedep_codecommune", "reg_id", "dep_code", "com_code"]].merge(
    communes_df[["id_codedep_codecommune", "COM", "PTOT"]],
    on="id_codedep_codecommune",# Clé commune pour la jointure
    how="left"                  # Jointure gauche pour conserver toutes les lignes de geo_df
)

# On renomme les colonnes pour les rendre compréhensibles
commune_df.columns = ["id_codedep_codecommune", "id_region", "code_departement", "code_commune", "nom_commune", "nbre_habitant_2019"]

# Export du fichier COMMUNE
commune_df.to_csv("COMMUNE.csv", index=False, encoding='utf-8-sig', sep=';')


In [18]:
 # On renomme les colonnes pour les rendre compréhensibles
communes_df.columns = ["id_codedep_codecommune", "id_region", "code_commune", "Code_departement", "nbre_habitant_2019","nom_commune"]
communes_df[:15000].to_csv("COMMUNE1.csv", index=False, encoding='utf-8-sig', sep=';')
communes_df[15000:].to_csv("COMMUNE2.csv", index=False, encoding='utf-8-sig', sep=';')


In [20]:
communes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34991 entries, 0 to 34990
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id_codedep_codecommune  34991 non-null  object
 1   id_region               34991 non-null  int64 
 2   code_commune            34991 non-null  object
 3   Code_departement        34991 non-null  object
 4   nbre_habitant_2019      34991 non-null  int64 
 5   nom_commune             34991 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.6+ MB


In [22]:
communes_df.id_codedep_codecommune.apply(lambda x:len(x))

0        5
1        5
2        5
3        5
4        5
        ..
34986    6
34987    6
34988    6
34989    6
34990    6
Name: id_codedep_codecommune, Length: 34991, dtype: int64

In [24]:
communes_df['Code_departement'].str.startswith(('91', '92', '93', '94', '95')).sum()


501

In [26]:
communes_df[communes_df['Code_departement'].str[:2].isin(['91', '92', '93', '94', '95'])]


,id_codedep_codecommune,id_region,code_commune,Code_departement,nbre_habitant_2019,nom_commune
34377,91001,11,001,91,332,Abbéville-la-Rivière
34378,91016,11,016,91,4423,Angerville
34379,91017,11,017,91,1749,Angervilliers
34380,91021,11,021,91,11447,Arpajon
34381,91022,11,022,91,140,Arrancourt
...,...,...,...,...,...,...
34873,95676,11,676,95,500,Villers-en-Arthies
34874,95678,11,678,95,872,Villiers-Adam
34875,95680,11,680,95,28540,Villiers-le-Bel
34876,95682,11,682,95,192,Villiers-le-Sec


In [28]:
valeurs_df

,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Nom de l'acquereur,id_codedep_codecommune
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-02,Vente,...,2,Appartement,NaN,48,3,NaN,NaN,NaN,GUIRAO,01103
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-02,Vente,...,2,Appartement,NaN,40,1,NaN,NaN,NaN,HARNOIS,06004
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-02,Vente,...,2,Appartement,NaN,82,3,NaN,NaN,NaN,ROGIER,06088
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-02,Vente,...,2,Appartement,NaN,27,1,NaN,NaN,NaN,BOCQUIER,06123
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-02,Vente,...,2,Appartement,NaN,47,2,NaN,NaN,NaN,GUILLOSSOU,13005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-06-30,Vente,...,2,Appartement,NaN,63,3,NaN,NaN,NaN,SAADOUN,95585
34165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-06-30,Vente,...,2,Appartement,NaN,66,3,NaN,NaN,NaN,GUECHI,95598
34166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-06-30,Vente,...,2,Appartement,NaN,63,3,NaN,NaN,NaN,PEDRON,95607
34167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-06-30,Vente,...,2,Appartement,NaN,77,4,NaN,NaN,NaN,HOUARD,972213


In [30]:
communes_df

,id_codedep_codecommune,id_region,code_commune,Code_departement,nbre_habitant_2019,nom_commune
0,01001,84,001,01,798,L'Abergement-Clémenciat
1,01002,84,002,01,257,L'Abergement-de-Varey
2,01004,84,004,01,14514,Ambérieu-en-Bugey
3,01005,84,005,01,1776,Ambérieux-en-Dombes
4,01006,84,006,01,118,Ambléon
...,...,...,...,...,...,...
34986,974420,4,420,974,24292,Sainte-Suzanne
34987,974421,4,421,974,7209,Salazie
34988,974422,4,422,974,80833,Le Tampon
34989,974423,4,423,974,7106,Les Trois-Bassins


**Étape 5 : Sélectionner les colonnes pour la table BIEN**

In [33]:
# On sélectionne les colonnes utiles à propos des biens immobiliers
bien_df = valeurs_df[[
    "Identifiant local", "id_codedep_codecommune", "No voie", "B/T/Q",
    "Type de voie", "Voie", "Nombre pieces principales", 
    "Surface Carrez du 1er lot", "Surface reelle bati", "Type local"
]].copy()

# On renomme les colonnes pour la table BIEN
bien_df.columns = [
    "id_bien", "id_codedep_codecommune", "no_voie", "BTQ", "type_voie",
    "voie", "total_piece", "surface_carrez", "surface_local", "type_local"
]

# Export du fichier BIEN
bien_df.to_csv("BIEN.csv", index=False, encoding='utf-8-sig', sep=';')



In [35]:
print(bien_df.memory_usage(deep=True).sum() / (1024 * 1024), "MB")


9.757880210876465 MB


In [37]:
# Assigner un ID numérique au unique_id de BIEN
bien_df['id_bien'] = range(1, len(bien_df) + 1)


In [39]:
# Define the columns to concatenate
columns_to_concat = ['type_voie', 'no_voie', 'voie','BTQ','total_piece','surface_carrez','type_local']  # Replace with your actual column names

# Create a new column 'unique_id' by concatenating the specified columns
bien_df['unique_id'] =bien_df[columns_to_concat].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

# Display the first few rows to verify
print(bien_df[['unique_id']].head())

                                           unique_id
0       RUE_347.0_DU CHATEAU_nan_3_48.22_Appartement
1     BD_4.0_EDOUARD BAUDOIN_nan_1_39.11_Appartement
2             RUE_20.0_MARCEAU_B_3_80.25_Appartement
3  RTE_550.0_DES VESPINS RN7_nan_1_27.51_Appartement
4  RES_9300.0_LES ARPEGES BD DES ABA_nan_2_47.33_...


**Étape 8 : Exporter le fichier BIEN.csv**

In [42]:
# 1.6 Exporter
bien_df.to_csv("BIEN.csv", index=False)
print("→ BIEN.csv:", bien_df.shape)

→ BIEN.csv: (34169, 11)


In [44]:
bien_df

,id_bien,id_codedep_codecommune,no_voie,BTQ,type_voie,voie,total_piece,surface_carrez,surface_local,type_local,unique_id
0,1,01103,347.0,NaN,RUE,DU CHATEAU,3,48.22,48,Appartement,RUE_347.0_DU CHATEAU_nan_3_48.22_Appartement
1,2,06004,4.0,NaN,BD,EDOUARD BAUDOIN,1,39.11,40,Appartement,BD_4.0_EDOUARD BAUDOIN_nan_1_39.11_Appartement
2,3,06088,20.0,B,RUE,MARCEAU,3,80.25,82,Appartement,RUE_20.0_MARCEAU_B_3_80.25_Appartement
3,4,06123,550.0,NaN,RTE,DES VESPINS RN7,1,27.51,27,Appartement,RTE_550.0_DES VESPINS RN7_nan_1_27.51_Appartement
4,5,13005,9300.0,NaN,RES,LES ARPEGES BD DES ABA,2,47.33,47,Appartement,RES_9300.0_LES ARPEGES BD DES ABA_nan_2_47.33_...
...,...,...,...,...,...,...,...,...,...,...,...
34164,34165,95585,5.0,NaN,PL,JEAN CHARCOT,3,59.50,63,Appartement,PL_5.0_JEAN CHARCOT_nan_3_59.5_Appartement
34165,34166,95598,59.0,NaN,AV,DE PARIS,3,55.29,66,Appartement,AV_59.0_DE PARIS_nan_3_55.29_Appartement
34166,34167,95607,4.0,NaN,RUE,RENE ECHAVIDRE,3,62.05,63,Appartement,RUE_4.0_RENE ECHAVIDRE_nan_3_62.05_Appartement
34167,34168,972213,5919.0,F,RES,L EMERAUDE MORNE PAVILLON,4,77.88,77,Appartement,RES_5919.0_L EMERAUDE MORNE PAVILLON_F_4_77.88...


In [46]:
#Afficher la liste des colonnes
print(bien_df.columns.tolist())


['id_bien', 'id_codedep_codecommune', 'no_voie', 'BTQ', 'type_voie', 'voie', 'total_piece', 'surface_carrez', 'surface_local', 'type_local', 'unique_id']


In [48]:
#Supprimer les doublons sur unique_id
bien_df=bien_df.drop_duplicates('unique_id')

In [50]:
#Vérifier l’unicité de chaque bien
bien_df['unique_id'].value_counts()

unique_id
RUE_347.0_DU CHATEAU_nan_3_48.22_Appartement             1
RUE_42.0_DE LA BERGERIE_nan_3_58.46_Appartement          1
RUE_26.0_MONTROSIER_nan_3_111.08_Appartement             1
ALL_19.0_DU TERTRE_nan_6_124.41_Maison                   1
RUE_44.0_PERIER_nan_3_46.1_Appartement                   1
                                                        ..
RUE_42.0_DU FAUBOURG DU TEMPLE_nan_4_61.4_Appartement    1
RUE_17.0_DE PARADIS_B_3_100.94_Appartement               1
RUE_21.0_D ALSACE_nan_3_170.66_Appartement               1
RUE_38.0_LOUIS BLANC_nan_2_46.31_Appartement             1
RES_5368.0_GRAND VILLAGE_F_4_75.7_Appartement            1
Name: count, Length: 34153, dtype: int64

In [52]:
#Filtrer un bien spécifique par son identifiant
bien_df[bien_df['unique_id']=='RUE_347.0_DU CHATEAU_nan_3_48.22_Appartement']

,id_bien,id_codedep_codecommune,no_voie,BTQ,type_voie,voie,total_piece,surface_carrez,surface_local,type_local,unique_id
0,1,01103,347.0,NaN,RUE,DU CHATEAU,3,48.22,48,Appartement,RUE_347.0_DU CHATEAU_nan_3_48.22_Appartement


In [54]:
bien_df.columns

Index(['id_bien', 'id_codedep_codecommune', 'no_voie', 'BTQ', 'type_voie',
       'voie', 'total_piece', 'surface_carrez', 'surface_local', 'type_local',
       'unique_id'],
      dtype='object')

In [56]:
print(bien_df.columns.tolist())


['id_bien', 'id_codedep_codecommune', 'no_voie', 'BTQ', 'type_voie', 'voie', 'total_piece', 'surface_carrez', 'surface_local', 'type_local', 'unique_id']


In [58]:
#Vérifier le type de données de unique_id
print(type(bien_df['unique_id'].iloc[0]))


<class 'str'>


In [60]:

columns_to_concat = ["Type de voie", "No voie", "Voie", "B/T/Q", "Nombre pieces principales", "Surface Carrez du 1er lot", "Type local"]
#valeurs_df['unique_id'] = valeurs_df[columns_to_concat] \
                            #.fillna('') \
                            #.astype(str) \
                            #.agg('_'.join, axis=1)
valeurs_df['unique_id'] =valeurs_df[columns_to_concat].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)


In [62]:
vente_df = valeurs_df[['unique_id', 'Date mutation', 'Valeur fonciere']].copy()
vente_df.columns = ['unique_id', 'date', 'valeur']


In [66]:
vente_df

,unique_id,date,valeur
0,RUE_347.0_DU CHATEAU_nan_3_48.22_Appartement,2020-01-02,165000.0
1,BD_4.0_EDOUARD BAUDOIN_nan_1_39.11_Appartement,2020-01-02,355680.0
2,RUE_20.0_MARCEAU_B_3_80.25_Appartement,2020-01-02,229500.0
3,RTE_550.0_DES VESPINS RN7_nan_1_27.51_Appartement,2020-01-02,125000.0
4,RES_9300.0_LES ARPEGES BD DES ABA_nan_2_47.33_...,2020-01-02,90000.0
...,...,...,...
34164,PL_5.0_JEAN CHARCOT_nan_3_59.5_Appartement,2020-06-30,115000.0
34165,AV_59.0_DE PARIS_nan_3_55.29_Appartement,2020-06-30,129000.0
34166,RUE_4.0_RENE ECHAVIDRE_nan_3_62.05_Appartement,2020-06-30,212000.0
34167,RES_5919.0_L EMERAUDE MORNE PAVILLON_F_4_77.88...,2020-06-30,220000.0


In [68]:
bien_df.unique_id.isin(vente_df.unique_id).value_counts()

unique_id
True    34153
Name: count, dtype: int64

In [70]:
vente_df = vente_df.merge(bien_df[['id_bien', 'unique_id']], on='unique_id', how='inner')

In [72]:
vente_df

,unique_id,date,valeur,id_bien
0,RUE_347.0_DU CHATEAU_nan_3_48.22_Appartement,2020-01-02,165000.0,1
1,BD_4.0_EDOUARD BAUDOIN_nan_1_39.11_Appartement,2020-01-02,355680.0,2
2,RUE_20.0_MARCEAU_B_3_80.25_Appartement,2020-01-02,229500.0,3
3,RTE_550.0_DES VESPINS RN7_nan_1_27.51_Appartement,2020-01-02,125000.0,4
4,RES_9300.0_LES ARPEGES BD DES ABA_nan_2_47.33_...,2020-01-02,90000.0,5
...,...,...,...,...
34164,PL_5.0_JEAN CHARCOT_nan_3_59.5_Appartement,2020-06-30,115000.0,34165
34165,AV_59.0_DE PARIS_nan_3_55.29_Appartement,2020-06-30,129000.0,34166
34166,RUE_4.0_RENE ECHAVIDRE_nan_3_62.05_Appartement,2020-06-30,212000.0,34167
34167,RES_5919.0_L EMERAUDE MORNE PAVILLON_F_4_77.88...,2020-06-30,220000.0,34168


In [74]:
# Supprimer la colonne "unique_id" car elle n'est plus utile
vente_df = vente_df.drop(columns=['unique_id'])

In [76]:
# Créer un identifiant unique pour chaque vente (id_vente)
vente_df['id_vente'] = range(1, len(vente_df) + 1)

In [78]:
# Ne conserver que les colonnes utiles pour l'analyse ou la base finale
vente_df = vente_df[['id_vente', 'id_bien', 'date', 'valeur']]

In [80]:
# Convertir les dates au format datetime (interpréter comme jour/mois/année)
vente_df['date'] = pd.to_datetime(vente_df['date'], format='%d/%m/%Y')  # lire comme jour/mois/année
# Reformater les dates au format ISO (année-mois-jour)
vente_df['date'] = vente_df['date'].dt.strftime('%Y-%m-%d')             # écrire comme année-mois-jour

In [82]:
# Sauvegarder le fichier de ventes sous forme CSV, encodage compatible Excel
vente_df.to_csv("VENTE.csv", index=False, encoding='utf-8-sig', sep=';')
print("→ VENTE.csv:", vente_df.shape)

→ VENTE.csv: (34169, 4)


In [84]:
vente_df.head

<bound method NDFrame.head of        id_vente  id_bien        date    valeur
0             1        1  2020-01-02  165000.0
1             2        2  2020-01-02  355680.0
2             3        3  2020-01-02  229500.0
3             4        4  2020-01-02  125000.0
4             5        5  2020-01-02   90000.0
...         ...      ...         ...       ...
34164     34165    34165  2020-06-30  115000.0
34165     34166    34166  2020-06-30  129000.0
34166     34167    34167  2020-06-30  212000.0
34167     34168    34168  2020-06-30  220000.0
34168     34169    34169  2020-06-30  121000.0

[34169 rows x 4 columns]>

In [86]:
vente_df['id_bien'].value_counts()

id_bien
3171     3
6829     2
13493    2
15492    2
2853     2
        ..
11385    1
11384    1
11383    1
11382    1
34169    1
Name: count, Length: 34153, dtype: int64

In [88]:
print(bien_df[['id_bien']].head())


   id_bien
0        1
1        2
2        3
3        4
4        5


In [90]:
vente_df.head()

,id_vente,id_bien,date,valeur
0,1,1,2020-01-02,165000.0
1,2,2,2020-01-02,355680.0
2,3,3,2020-01-02,229500.0
3,4,4,2020-01-02,125000.0
4,5,5,2020-01-02,90000.0


In [92]:
# 3.1 Sélection + suppression des doublons
region_df = geo_df[["reg_code", "reg_nom", "regrgp_nom"]].drop_duplicates()


In [94]:
# 3.2 Renommer
region_df.columns = ["id_region", "nom_region", "nom_regroup"]


In [96]:
# 3.3 Exporter
region_df.to_csv("REGION.csv", index=False,encoding='utf-8-sig', sep=';')
print("→ REGION.csv:", region_df.shape)

→ REGION.csv: (19, 3)


In [98]:
# Afficher les valeurs qui ne ressemblent pas à une date
problemes = vente_df[~vente_df['date'].str.contains(r'\d{2}/\d{2}/\d{4}', na=False)]
print(problemes)


       id_vente  id_bien        date    valeur
0             1        1  2020-01-02  165000.0
1             2        2  2020-01-02  355680.0
2             3        3  2020-01-02  229500.0
3             4        4  2020-01-02  125000.0
4             5        5  2020-01-02   90000.0
...         ...      ...         ...       ...
34164     34165    34165  2020-06-30  115000.0
34165     34166    34166  2020-06-30  129000.0
34166     34167    34167  2020-06-30  212000.0
34167     34168    34168  2020-06-30  220000.0
34168     34169    34169  2020-06-30  121000.0

[34169 rows x 4 columns]


In [100]:
# Les id_bien qui existent dans la table BIEN
biens = bien_df['id_bien'].unique()

# Les ventes qui ont un id_bien non présent dans BIEN
problemes = vente_df[~vente_df['id_bien'].isin(biens)]

print(problemes)


Empty DataFrame
Columns: [id_vente, id_bien, date, valeur]
Index: []


In [102]:
doublons = bien_df[bien_df.duplicated(subset='id_bien', keep=False)]
print(doublons)


Empty DataFrame
Columns: [id_bien, id_codedep_codecommune, no_voie, BTQ, type_voie, voie, total_piece, surface_carrez, surface_local, type_local, unique_id]
Index: []


In [104]:
print(bien_df['id_bien'].duplicated().sum())


0


In [106]:
bien_df = pd.read_csv("BIEN.csv", sep=';', encoding='utf-8-sig')


In [108]:
print(bien_df.columns.tolist())


['id_bien,id_codedep_codecommune,no_voie,BTQ,type_voie,voie,total_piece,surface_carrez,surface_local,type_local,unique_id']


In [110]:
bien_df.columns = bien_df.columns.str.strip()  # Supprime les espaces
bien_df.columns = bien_df.columns.str.lower()  # Met tout en minuscules


In [122]:
# 3. Vérifier rapidement
print(bien_df.dtypes)
print(bien_df.head())

id_bien,id_codedep_codecommune,no_voie,btq,type_voie,voie,total_piece,surface_carrez,surface_local,type_local,unique_id    object
dtype: object
  id_bien,id_codedep_codecommune,no_voie,btq,type_voie,voie,total_piece,surface_carrez,surface_local,type_local,unique_id
0  1,01103,347.0,,RUE,DU CHATEAU,3,48.22,48,Appar...                                                                     
1  2,06004,4.0,,BD,EDOUARD BAUDOIN,1,39.11,40,App...                                                                     
2  3,06088,20.0,B,RUE,MARCEAU,3,80.25,82,Appartem...                                                                     
3  4,06123,550.0,,RTE,DES VESPINS RN7,1,27.51,27,...                                                                     
4  5,13005,9300.0,,RES,LES ARPEGES BD DES ABA,2,4...                                                                     


In [124]:
# 4. Sauvegarder un fichier BIEN_CLEAN.csv
bien_df.to_csv("BIEN.csv", sep=';', index=False, encoding='utf-8')

print("Fichier BIEN.csv prêt pour MySQL !")

Fichier BIEN.csv prêt pour MySQL !


In [126]:
print(bien_df.head())

  id_bien,id_codedep_codecommune,no_voie,btq,type_voie,voie,total_piece,surface_carrez,surface_local,type_local,unique_id
0  1,01103,347.0,,RUE,DU CHATEAU,3,48.22,48,Appar...                                                                     
1  2,06004,4.0,,BD,EDOUARD BAUDOIN,1,39.11,40,App...                                                                     
2  3,06088,20.0,B,RUE,MARCEAU,3,80.25,82,Appartem...                                                                     
3  4,06123,550.0,,RTE,DES VESPINS RN7,1,27.51,27,...                                                                     
4  5,13005,9300.0,,RES,LES ARPEGES BD DES ABA,2,4...                                                                     


In [128]:
commune_df = pd.read_csv("COMMUNE.csv", sep=';',  encoding='utf-8-sig')


In [130]:
print(commune_df.columns.tolist())

['id_codedep_codecommune', 'id_region', 'code_departement', 'code_commune', 'nom_commune', 'nbre_habitant_2019']


In [132]:
print(commune_df.head())

  id_codedep_codecommune id_region code_departement code_commune  nom_commune  \
0                0101001       R84               01        01001          NaN   
1                0101002       R84               01        01002          NaN   
2                0101003       R84               01        01003          NaN   
3                0101004       R84               01        01004          NaN   
4                0101005       R84               01        01005          NaN   

   nbre_habitant_2019  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  


In [134]:
print(commune_df.isnull().sum())

id_codedep_codecommune        0
id_region                     0
code_departement              0
code_commune                  0
nom_commune               38916
nbre_habitant_2019        38916
dtype: int64


In [136]:
print(commune_df.dtypes)

id_codedep_codecommune     object
id_region                  object
code_departement           object
code_commune               object
nom_commune               float64
nbre_habitant_2019        float64
dtype: object


In [138]:
print(commune_df.columns)   # Regarde les colonnes exactes


Index(['id_codedep_codecommune', 'id_region', 'code_departement',
       'code_commune', 'nom_commune', 'nbre_habitant_2019'],
      dtype='object')


In [140]:
commune_df = commune_df.rename(columns={
    'id_code_dep_codecommune': 'id_code_commune',  # Je simplifie le nom
    'code_commune': 'code_commune',
    'Code_departement': 'code_departement',
    'nbre_habitant_2019': 'nbre_habitant_2019',
    'Nom_commune': 'nom_commune',
    'id_region': 'id_region'
})

In [142]:
print(commune_df.columns.tolist())

['id_codedep_codecommune', 'id_region', 'code_departement', 'code_commune', 'nom_commune', 'nbre_habitant_2019']


In [144]:
commune_df.columns = commune_df.columns.str.strip()

In [146]:
print(commune_df.columns.tolist())

['id_codedep_codecommune', 'id_region', 'code_departement', 'code_commune', 'nom_commune', 'nbre_habitant_2019']


In [148]:
commune_df.to_csv("COMMUNE.csv", sep=';', index=False, encoding='utf-8')

In [150]:
import pandas as pd
vente_df = pd.read_csv("VENTE.csv", sep=";", encoding="utf-8-sig")
print(vente_df.columns.tolist())


['id_vente', 'id_bien', 'date', 'valeur']


In [152]:
# Affiche les types
print(vente_df.dtypes)

# Vérifie les valeurs manquantes
print(vente_df.isnull().sum())

# Affiche un échantillon
print(vente_df.head())

id_vente      int64
id_bien       int64
date         object
valeur      float64
dtype: object
id_vente     0
id_bien      0
date         0
valeur      18
dtype: int64
   id_vente  id_bien        date    valeur
0         1        1  2020-01-02  165000.0
1         2        2  2020-01-02  355680.0
2         3        3  2020-01-02  229500.0
3         4        4  2020-01-02  125000.0
4         5        5  2020-01-02   90000.0


In [154]:
# Convertir la colonne date en format datetime
vente_df['date'] = pd.to_datetime(vente_df['date'], format="%Y-%m-%d", errors='coerce')

# Vérifions maintenant
print(vente_df.dtypes)
print(vente_df.isnull().sum())
print(vente_df.head())


id_vente             int64
id_bien              int64
date        datetime64[ns]
valeur             float64
dtype: object
id_vente     0
id_bien      0
date         0
valeur      18
dtype: int64
   id_vente  id_bien       date    valeur
0         1        1 2020-01-02  165000.0
1         2        2 2020-01-02  355680.0
2         3        3 2020-01-02  229500.0
3         4        4 2020-01-02  125000.0
4         5        5 2020-01-02   90000.0


In [156]:
# Sauvegarder le fichier VENTE propre
vente_df.to_csv("VENTE.csv", sep=",", index=False, encoding="utf-8-sig")
print("✅ Fichier VENTE.csv sauvegardé avec succès!")


✅ Fichier VENTE.csv sauvegardé avec succès!


In [158]:
# Recharger REGION.csv correctement en forçant la séparation
region_df = pd.read_csv("REGION.csv", sep=",", encoding="utf-8-sig")

# Corriger les types si besoin
region_df['id_region'] = region_df['id_region'].astype(int)

# Vérifier après correction
print(region_df.columns.tolist())
print(region_df.dtypes)
print(region_df.head())
print(region_df.isnull().sum())

# Sauvegarder dans un nouveau fichier propre
region_df.to_csv("REGION.csv", index=False, encoding="utf-8-sig")


KeyError: 'id_region'

In [160]:
print(communes_df.id_codedep_codecommune.dtype)
print(bien_df.id_codedep_codecommune.dtype)


object


AttributeError: 'DataFrame' object has no attribute 'id_codedep_codecommune'

In [162]:
communes_df.id_codedep_codecommune.isin(bien_df.id_codedep_codecommune).value_counts()

AttributeError: 'DataFrame' object has no attribute 'id_codedep_codecommune'

In [164]:
bien_df.id_codedep_codecommune.isin(communes_df.id_codedep_codecommune).value_counts()

AttributeError: 'DataFrame' object has no attribute 'id_codedep_codecommune'

In [166]:
missing_codes = communes_df.loc[~communes_df.id_codedep_codecommune.isin(bien_df.id_codedep_codecommune), 'id_codedep_codecommune']
print(missing_codes.unique()[:20])


AttributeError: 'DataFrame' object has no attribute 'id_codedep_codecommune'